# Summarize Walktrap Clusters

In [1]:
# modify these for your own computer
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import glob
import cPickle  as pickle
from collections import OrderedDict
import json


# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami
from sklearn.metrics import mutual_info_score as mi
from sklearn.metrics import adjusted_rand_score as ar
from sklearn.metrics import calinski_harabaz_score as ch # (X, labels)
from sklearn.metrics import completeness_score as cs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import fowlkes_mallows_score as fm
from sklearn.metrics import homogeneity_completeness_v_measure as hcvm
from sklearn.metrics import homogeneity_score as hs # metric isn't symmetric (labels_true, labels_predicted)
from sklearn.metrics import silhouette_score as ss # (X, labels)
from sklearn.metrics import silhouette_samples as ss2 # (X, labels)
from sklearn.metrics import v_measure_score as vm

import scipy.sparse
import random
import itertools
from itertools import combinations


# our code
sys.path.append(repo_directory + 'code/')
from summarize_clusters import *
from helpful_functions import *

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)
nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# csv location
csv_dir = "C:/Users/Michael/Documents/GitHub/law-net/csv/"
csv_dir_mod = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_modularity/"
csv_dir_walk = "C:/Users/Michael/Documents/GitHub/law-net/csv/summarize_walktrap/"

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')

# all opinions
all_the_opinions = all_opinions(file_paths)

# clusters directory
clusters_dir = "C:/Users/Michael/Desktop/network_data/raw/scotus/clusters/"

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

## load tf-idf vectors
**tfidf_matrix** = (row_index, column_index): tf_idf value (**CSR FORMAT**)  
**op_id_to_bow_id** = opinion_id (corresponds to row indices)  
**vocab** = all the words in tfidf_matrix (correspond to column indices)

In [2]:
tfidf_matrix, op_id_to_bow_id, vocab = load_tf_idf(nlp_sub_dir)

# Clustering Work:
focus on largest connected component of **undirected scotus**

In [3]:
# load the graph
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

# limit ourselves to cases upto and including 2015 since we are missing some textfiles from 2016
G = G.subgraph(G.vs.select(year_le=2015))

# make graph undirected
Gud = G.copy()
Gud = Gud.as_undirected()

# get largest connected componenet
components = Gud.clusters(mode='STRONG')
g = components.subgraphs()[np.argmax(components.sizes())]

# CL ids of cases in largest connected component
CLids = g.vs['name']

## walktrap on undirected scotus

In [4]:
%%time

# walktrap clustering
cd_walktrap = g.community_walktrap()

wt_clust = cd_walktrap.as_clustering()

print wt_clust.summary()

# save clusters in pandas
walktrap_clusters = pd.Series(wt_clust.membership, index=g.vs['name'])

Clustering with 24724 elements and 2264 clusters
Wall time: 2min 49s


## get the top 5 biggest clusters

In [5]:
'''
dict_top_n_clusters = dictionary of top K clusters 
                      (key=cluster #, value=opinions in cluster)
                      
biggest_n_clusters = list of top K clusters (int)
'''

dict_top_n_clusters, biggest_n_clusters = get_top_n_clusters(5, len(wt_clust), walktrap_clusters)

cluster 1 : 3851 opinions
cluster 2 : 2640 opinions
cluster 5 : 1996 opinions
cluster 3 : 1494 opinions
cluster 12 : 1103 opinions


# Top K Words of Each Cluster
This function summarizes a set of opinions by returning the words that appear in these opinions with the highest tf-idf scores.

# Top K Words ($\mu_{cluster}$) of Each Cluster
compute the mean tf-idf vector of the cluster, return the top K words from this mean vector

# Top K Words ($\mu_{cluster} - \mu_{complement}$ ) of Each Cluster
compute the mean tf-idf vector of the cluster and also of the complement of the cluster,  
take the difference mu_cluster - mu_complement, return the top K words in this difference

# Most Relevant Opinion of Each Cluster
compute the mean tf-idf vector, return the document in the cluster closet to the mean  

# Cluster 1 Summary (3851 opinions)

In [7]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[1]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_walk + "cluster_1.csv")

print cluster_info.shape

(3851, 3)
Wall time: 2.41 s


In [98]:
cluster_info = pd.read_csv(csv_dir_walk + 'cluster_1.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Dalton v. Jennings,1876-12-18,https://www.courtlistener.com/opinion/89374/da...
1,1,Windsor v. McVeigh,1876-12-11,https://www.courtlistener.com/opinion/89375/wi...
2,2,Bigelow v. Berkshire Life Ins. Co.,1876-12-11,https://www.courtlistener.com/opinion/89376/bi...
3,3,Indianapolis & St. Louis R. Co. v. Horst,1876-12-18,https://www.courtlistener.com/opinion/89378/in...
4,4,"The"" Atlas""",1876-11-27,https://www.courtlistener.com/opinion/89379/th...


In [8]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[1], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[1], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[1], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[1], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_walk + "cluster_1_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 43.6 s


In [19]:
cluster_summary = pd.read_csv(csv_dir_walk + 'cluster_1_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,guam,court,§,96819
1,1,wenzel,state,feder,96819
2,2,fslic,v,congress,96819
3,3,artwar,§,act,96819
4,4,bail,act,commiss,96819


# Cluster 2 Summary (2640 opinions)

In [10]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[2]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_walk + "cluster_2.csv")

print cluster_info.shape

(2640, 3)
Wall time: 1.57 s


In [11]:
cluster_info = pd.read_csv(csv_dir_walk + 'cluster_2.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Hamilton v. Alabama,1961-11-13,https://www.courtlistener.com/opinion/106300/h...
1,1,Hoyt v. Florida,1961-11-20,https://www.courtlistener.com/opinion/106302/h...
2,2,Hodges v. United States,1961-12-04,https://www.courtlistener.com/opinion/106305/h...
3,3,Alaska Packers Assn. v. Pillsbury,1937-04-26,https://www.courtlistener.com/opinion/102813/a...
4,4,Bradshaw v. Stumpf,2005-06-13,https://www.courtlistener.com/opinion/799973/b...


In [12]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[2], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[2], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[2], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[2], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_walk + "cluster_2_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 29.2 s


In [13]:
cluster_summary = pd.read_csv(csv_dir_walk + 'cluster_2_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,baal,court,sentenc,108329
1,1,stumpf,state,convict,108329
2,2,passport,v,trial,108329
3,3,lagrand,petition,juri,108329
4,4,ree,sct,petition,108329


# Cluster 5 Summary (1996 opinions)

In [14]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[5]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_walk + "cluster_5.csv")

print cluster_info.shape

(1996, 3)
Wall time: 1.2 s


In [15]:
cluster_info = pd.read_csv(csv_dir_walk + 'cluster_5.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Windsor v. McVeigh,1876-12-11,https://www.courtlistener.com/opinion/89375/wi...
1,1,California Medical Assn. v. Federal Election C...,1981-06-26,https://www.courtlistener.com/opinion/110551/c...
2,2,Kilbourn v. Thompson,1881-02-28,https://www.courtlistener.com/opinion/90311/ki...
3,3,Hawke v. Smith (No. 1),1920-06-01,https://www.courtlistener.com/opinion/99607/ha...
4,4,Regents of Univ. of Mich. v. Ewing,1985-12-12,https://www.courtlistener.com/opinion/111549/r...


In [17]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[5], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[5], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[5], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[5], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_walk + "cluster_5_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 28.6 s


In [18]:
cluster_summary = pd.read_csv(csv_dir_walk + 'cluster_5_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,lesag,court,led2d,104135
1,1,anastaplo,state,sct,104135
2,2,church,v,school,104135
3,3,flag,sct,us,104135
4,4,communist,led2d,speech,104135


# Cluster 3 Summary (1494 opinions)

In [20]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[3]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_walk + "cluster_3.csv")

print cluster_info.shape

(1494, 3)
Wall time: 859 ms


In [21]:
cluster_info = pd.read_csv(csv_dir_walk + 'cluster_3.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Barkley v. Levee Commissioners,1876-12-18,https://www.courtlistener.com/opinion/89372/ba...
1,1,Broughton v. Pensacola,1876-12-18,https://www.courtlistener.com/opinion/89373/br...
2,2,Atlantic Coast Line R. Co. v. Mazursky,1910-02-21,https://www.courtlistener.com/opinion/97158/at...
3,3,King v. West Virginia,1910-01-31,https://www.courtlistener.com/opinion/97154/ki...
4,4,Houston v. Southwestern Bell Telephone Co.,1922-05-29,https://www.courtlistener.com/opinion/100015/h...


In [22]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[3], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[3], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[3], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[3], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_walk + "cluster_3_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 17.9 s


In [23]:
cluster_summary = pd.read_csv(csv_dir_walk + 'cluster_3_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,vetterlein,state,compani,95657
1,1,vaccin,court,citi,95657
2,2,milk,compani,contract,95657
3,3,vs,v,ct,95657
4,4,sewer,contract,ordin,95657


# Cluster 12 Summary (1103 opinions)

In [24]:
%%time

opinion_names = []
opinion_dates = []
opinion_links = []

for i in dict_top_n_clusters[12]:
    try:
        with open(clusters_dir + i + ".json") as data_file:
            data = json.load(data_file)
    except IOError:
        pass
        #name, date, link = case_info2(i)
        #opinion_names.append(name)
        #opinion_dates.append(date)
        #opinion_links.append(link)
        
    name = data['case_name'].encode('utf-8')
    date = data['date_filed'].encode('utf-8')
    link = 'https://www.courtlistener.com' + data['absolute_url'].encode('utf-8')
    
    opinion_names.append(name)
    opinion_dates.append(date)
    opinion_links.append(link)
    
cluster_info = pd.DataFrame()
cluster_info['names'] = opinion_names
cluster_info['dates'] = opinion_dates
cluster_info['url'] = opinion_links

cluster_info.to_csv(csv_dir_walk + "cluster_12.csv")

print cluster_info.shape

(1103, 3)
Wall time: 566 ms


In [25]:
cluster_info = pd.read_csv(csv_dir_walk + 'cluster_12.csv')
cluster_info.head()

,Unnamed: 0,names,dates,url
0,0,Alabama v. King & Boozer,1941-11-10,https://www.courtlistener.com/opinion/103545/a...
1,1,Curry v. United States,1941-11-10,https://www.courtlistener.com/opinion/103546/c...
2,2,Pullman Co. v. Kansas Ex Rel. Coleman,1910-01-31,https://www.courtlistener.com/opinion/97151/pu...
3,3,Watson v. State Comptroller of NY,1920-11-15,https://www.courtlistener.com/opinion/99649/wa...
4,4,Greenough v. Tax Assessors of Newport,1947-06-09,https://www.courtlistener.com/opinion/104437/g...


In [26]:
%%time

k=1000 # number of words to get

top_words = top_k_words(dict_top_n_clusters[12], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_mean = top_k_words_from_mean_vector(dict_top_n_clusters[12], k, tfidf_matrix, op_id_to_bow_id, vocab)
top_words_from_diff = top_k_words_from_difference(dict_top_n_clusters[12], all_the_opinions, k, tfidf_matrix, op_id_to_bow_id, vocab)
most_relev_op = document_closest_to_mean(dict_top_n_clusters[12], tfidf_matrix, op_id_to_bow_id)

top_words = [x.encode('utf-8') for x in top_words]
top_words_from_mean = [x.encode('utf-8') for x in top_words_from_mean]
top_words_from_diff = [x.encode('utf-8') for x in top_words_from_diff]

#print '\x1b[1;31m' + "Top K Words:" + '\x1b[0m', top_words
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster):" + '\x1b[0m', top_words_from_mean
#print '\x1b[1;31m' + "Top K Words (Mu_Cluster - Mu_Complement):" + '\x1b[0m', top_words_from_diff
#print '\x1b[1;31m' + "Most Relevent Opinion:" + '\x1b[0m', most_relev_op

cluster_summary = pd.DataFrame()
cluster_summary['top_words'] = top_words
cluster_summary['top_words_from_mean'] = top_words_from_mean
cluster_summary['top_words_from_diff'] = top_words_from_diff
cluster_summary['most_relev_op'] = most_relev_op

cluster_summary.to_csv(csv_dir_walk + "cluster_12_summary.csv")

print cluster_summary.shape

(1000, 4)
Wall time: 13.5 s


In [27]:
cluster_summary = pd.read_csv(csv_dir_walk + 'cluster_12_summary.csv')
cluster_summary.head()

,Unnamed: 0,top_words,top_words_from_mean,top_words_from_diff,most_relev_op
0,0,milk,tax,tax,88605
1,1,boom,state,state,88605
2,2,ethanol,commerc,commerc,88605
3,3,export,v,interst,88605
4,4,shrimp,interst,taxat,88605
